pip install pymssql 



In [1]:
import pandas as pd
import pymssql 

In [273]:
server="SQL-retail2.nikamed.local"
base="Retail2_shops"

In [488]:
sql="""
with mag_iskl as (
SELECT CAST(_IDRRef AS uniqueidentifier) AS Ссылка, --исключение техн склада
_IDRRef, 
_Code AS Код
FROM _Reference10
where _Code='0-002'
),
--------------------------------------исх продажи-----------------------
sales_ish as (
select 
       cast(dateadd(year, -2000, d21._date_time) as date) as Дата , 
       cast(d21._number as varchar(20))+'_'+ cast(dateadd(year, -2000, d21._date_time) as varchar(50)) as  'НомерЧекаККМ',  
       iif(e._EnumOrder=1,-1,1) as 'ВидОперации'
       ,iif(e._EnumOrder=1, -d21t._Fld6384, d21t._Fld6384 ) as 'Количество'
       ,iif(e._EnumOrder=1, -d21t._Fld6397, d21t._Fld6397) as 'Сумма'
              , cast(r10._idrref as uniqueidentifier) as 'ccМагазин'
       , iif(r2_10._Code is null, cast(r10._idrref as uniqueidentifier), 
                                  cast(r2_10._idrref as uniqueidentifier) ) as 'ccМагазинМотивационный'
from _Document21_VT6378 d21t
left outer join _Document21 d21 on d21t._Document21_IDRRef = d21._IDRRef /*исх чек*/
LEFT OUTER JOIN _reference47 r47 ON d21._Fld6331RRef = r47._idrref /*вид документа*/
LEFT OUTER JOIN _Enum352 e ON d21._Fld6333RRef = e._idrref /*вид документа в перечислениях чека ккм - 0-продажа, 1-возврат*/
left outer join _Reference10 r10 on d21._Fld6339RRef = r10._idrref /*shop*/
left outer join  _Document21 d2_21 on d21._Fld6353RRef=d2_21._idrref /*поиск мотив маг*/
left outer join _Reference10 r2_10 on d2_21._Fld6339RRef = r2_10._idrref /*shop*/
where d21._marked = 0 and d21._Posted = 1
and d21._date_time >= '08.01.4021' 
union all
select 
       cast(dateadd(year, -2000, d._date_time) as date) as 'Дата', 
        cast(d._number as varchar(20))+'_'+ cast(dateadd(year, -2000, d._date_time) as varchar(50)) as  'НомерЧекаККМ',      
        -1
       , -dt._Fld3984 as 'Количество'
       , -dt._Fld3988 as 'СуммаЧека'
       , cast(r10._idrref as uniqueidentifier) as 'ccМагазин'
       , iif(r2_10._Code is null, cast(r10._idrref as uniqueidentifier), cast(r2_10._idrref as uniqueidentifier) ) as 'ccМагазинМотивационный'
from _Document216_VT3980 dt
left outer join _Document216 d on dt._Document216_IDRRef = d._IDRRef /*исх возвратный чек*/
left outer join _Reference10 r10 on d._Fld3950RRef = r10._idrref /*shop*/
left outer join  _Reference172 r172  on dt._Fld4001RRef = r172._idrref
left outer join  _Reference23 r23  on dt._Fld3982RRef = r23._idrref
left outer join _Reference131 r131 on d._Fld11137RRef= r131._idrref --  склады
left outer join _Reference10 r2_10 on r131._Fld2686RRef = r2_10._idrref /*shop к складу*/
where d._marked = 0 and d._Posted = 1
and d._date_time >= '08.01.4021'
),
-------------суммарные продажи------------------------------------

sales_sum2 as(
SELECT 

Дата as date_ss, 
sum(Сумма) as Summa, 
ccМагазин, 
ccМагазинМотивационный,
ВидОперации 

FROM sales_ish
group by Дата,  ccМагазин,ccМагазинМотивационный,ВидОперации 
),
-------------c кол чеков------------------------------------
sales_sum as(
SELECT  
        date_ss, 

        ccМагазин, 
        ccМагазинМотивационный,
        sum(Summa) as Summa, 
        sum(ВидОперации) as Kol4
        
FROM sales_sum2

group by date_ss,  ccМагазин, ccМагазинМотивационный
)



SELECT 
cast(dateadd(year, -2000, i._fld10076) as date) AS date_ii,
sum(_fld10077) AS Трафик,
cast( r._Fld2907RRef as uniqueidentifier) AS cсМагазин,
sum(sales_sum.Summa) as Summa_ii,
sum(Kol4) as Kol4
FROM _InfoRg10074 i 


left outer JOIN _reference142 r ON i._Fld10075RRef = r._IDRRef
left outer join mag_iskl mi on r._Fld2907RRef=mi._IDRRef
left outer join sales_sum  on sales_sum.date_ss=(cast(dateadd(year, -2000, i._fld10076) as date)) 
        and sales_sum.ccМагазин=cast( r._Fld2907RRef as uniqueidentifier)

WHERE i._fld10076 > '08.01.4021'
AND _fld10077 > 0
and mi._IDRRef is null
group by cast(dateadd(year, -2000, i._fld10076) as date) , cast( r._Fld2907RRef as uniqueidentifier)
"""

In [489]:
def read_sql(sql,base, serv):
    #with pymssql.connect(server=serv ,database = base ,charset =  'cp1251') as  conn:
    with pymssql.connect(server=serv ,charset =  'cp1251',database = base ) as  conn:                  
    
        cursor = conn.cursor()  
        df = pd.read_sql( sql,conn)
    return df

In [490]:
%%time
df_tr=read_sql(sql,base, server)

Wall time: 3.89 s


In [491]:
sum(df_tr['Трафик'].head())


467.0

In [492]:
sum(df_tr['Kol4'].head(1000))

nan

In [493]:
df_tr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11436 entries, 0 to 11435
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   date_ii    11436 non-null  object 
 1   Трафик     11436 non-null  float64
 2   cсМагазин  11436 non-null  object 
 3   Summa_ii   11432 non-null  float64
 4   Kol4       11432 non-null  float64
dtypes: float64(3), object(2)
memory usage: 446.8+ KB


conn.close()

In [95]:
%%time
df_tr=read_sql(sql,base, server)


Wall time: 540 ms


In [96]:
sum(df_tr['Трафик'])

690385.0

In [458]:
df_tr.head()


,date_ii,Трафик,cсМагазин,Summa_ii,Kol4
0,2021-08-30,55.0,1500fa8c-015d-0443-11e9-82b89e2126a9,872794.0,26.0
1,2021-09-04,72.0,4a3ced86-7492-28df-11e6-dd56e64b7598,842890.0,0.0
2,2021-09-20,73.0,1500538f-015d-032a-11eb-1464e106fc37,1926944.0,32.0
3,2021-09-18,15.0,1500c98c-015d-0443-11e9-5f8f30006ccb,161940.0,12.0
4,2021-09-23,42.0,4a3cc085-7492-28df-11e6-137b34be108f,583395.0,23.0


In [97]:
df_tr


,date_ii,Трафик,cсМагазин
0,2021-09-16,70.0,4a3ca086-7492-28df-11e1-26e56017ec38
1,2021-09-03,37.0,1500578d-015d-0b43-11e9-c998961af1c0
2,2021-08-23,83.0,4a3ced86-7492-28df-11e6-dd56e64b7598
3,2021-09-03,107.0,4a3ccdba-7492-28df-11e4-5b50ef7829d1
4,2021-09-11,49.0,4a3cc085-7492-28df-11e6-137b34be108f
...,...,...,...
11424,2021-08-12,46.0,1500458f-015d-032a-11eb-0c760711c161
11425,2021-09-27,43.0,4a3c3cb8-7492-28df-11e7-778fe6fa7541
11426,2021-08-14,42.0,4a3cf985-7492-28df-11e5-2c544f5c615a
11427,2021-09-03,36.0,1500808c-015d-0443-11e9-29fddcf9d848


In [460]:
sql = """
with sales_ish as (
select 
       cast(dateadd(year, -2000, d21._date_time) as date) as Дата , 
       cast(d21._number as varchar(20))+'_'+ cast(dateadd(year, -2000, d21._date_time) as varchar(50)) as  'НомерЧекаККМ',  
       iif(e._EnumOrder=1,-1,1) as 'ВидОперации'
       ,iif(e._EnumOrder=1, -d21t._Fld6384, d21t._Fld6384 ) as 'Количество'
       ,iif(e._EnumOrder=1, -d21t._Fld6397, d21t._Fld6397) as 'Сумма'
              , cast(r10._idrref as uniqueidentifier) as 'ccМагазин'
       , iif(r2_10._Code is null, cast(r10._idrref as uniqueidentifier), 
                                  cast(r2_10._idrref as uniqueidentifier) ) as 'ccМагазинМотивационный'
from _Document21_VT6378 d21t
left outer join _Document21 d21 on d21t._Document21_IDRRef = d21._IDRRef /*исх чек*/
LEFT OUTER JOIN _reference47 r47 ON d21._Fld6331RRef = r47._idrref /*вид документа*/
LEFT OUTER JOIN _Enum352 e ON d21._Fld6333RRef = e._idrref /*вид документа в перечислениях чека ккм - 0-продажа, 1-возврат*/
left outer join _Reference10 r10 on d21._Fld6339RRef = r10._idrref /*shop*/
left outer join  _Document21 d2_21 on d21._Fld6353RRef=d2_21._idrref /*поиск мотив маг*/
left outer join _Reference10 r2_10 on d2_21._Fld6339RRef = r2_10._idrref /*shop*/
where d21._marked = 0 and d21._Posted = 1
and d21._date_time >= '08.01.4021' 
union all
select 
       cast(dateadd(year, -2000, d._date_time) as date) as 'Дата', 
        cast(d._number as varchar(20))+'_'+ cast(dateadd(year, -2000, d._date_time) as varchar(50)) as  'НомерЧекаККМ',      
        -1
       , -dt._Fld3984 as 'Количество'
       , -dt._Fld3988 as 'СуммаЧека'
       , cast(r10._idrref as uniqueidentifier) as 'ccМагазин'
       , iif(r2_10._Code is null, cast(r10._idrref as uniqueidentifier), cast(r2_10._idrref as uniqueidentifier) ) as 'ccМагазинМотивационный'
from _Document216_VT3980 dt
left outer join _Document216 d on dt._Document216_IDRRef = d._IDRRef /*исх возвратный чек*/
left outer join _Reference10 r10 on d._Fld3950RRef = r10._idrref /*shop*/
left outer join  _Reference172 r172  on dt._Fld4001RRef = r172._idrref
left outer join  _Reference23 r23  on dt._Fld3982RRef = r23._idrref
left outer join _Reference131 r131 on d._Fld11137RRef= r131._idrref --  склады
left outer join _Reference10 r2_10 on r131._Fld2686RRef = r2_10._idrref /*shop к складу*/
where d._marked = 0 and d._Posted = 1
and d._date_time >= '08.01.4021'
),
-------------суммарные продажи------------------------------------

sales_sum2 as(
SELECT Дата as date_ss, sum(Сумма) as Summa, ccМагазин, ccМагазинМотивационный,ВидОперации FROM sales_ish
group by Дата,  ccМагазин,ccМагазинМотивационный,ВидОперации 
)
-------------кол чеков------------------------------------
SELECT  date_ss, sum(Summa) as Summa, 
        ccМагазин, 
        ccМагазинМотивационный,
        sum(ВидОперации) as Kol4
        
FROM sales_sum2

group by date_ss,  ccМагазин,ccМагазинМотивационный

"""

In [461]:
def read_sql(sql,base, serv):
    #with pymssql.connect(server=serv ,database = base ,charset =  'cp1251') as  conn:
    with pymssql.connect(server=serv ,database = base,charset =  'cp1251' ) as  conn:                  
    
        cursor = conn.cursor()  
        df = pd.read_sql( sql,conn)
    return df

In [462]:
%%time
df=read_sql(sql,base, server)

Wall time: 6.8 s


In [466]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13175 entries, 0 to 13174
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   date_ss                 13175 non-null  object 
 1   Summa                   13175 non-null  float64
 2   ccМагазин               13175 non-null  object 
 3   ccМагазинМотивационный  13175 non-null  object 
 4   Kol4                    13175 non-null  int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 514.8+ KB


In [463]:
sum(df['Summa'])


1119209866.7600002

In [464]:
df

,date_ss,Summa,ccМагазин,ccМагазинМотивационный,Kol4
0,2021-08-04,82034.0,1500808c-015d-0443-11e9-29f23582b382,1500808c-015d-0443-11e9-29f23582b382,1
1,2021-08-07,126347.0,1500e38c-015d-0443-11e9-719a6d905a8c,1500e38c-015d-0443-11e9-719a6d905a8c,1
2,2021-08-29,80027.0,1500458f-015d-032a-11eb-0c9223d638c8,1500458f-015d-032a-11eb-0c9223d638c8,1
3,2021-08-26,59295.0,4a3cb0b9-7492-28df-11e8-164073d5498d,4a3cb0b9-7492-28df-11e8-164073d5498d,1
4,2021-09-27,45302.0,4a3c70b9-7492-28df-11e6-cc0c3dc73e03,4a3c70b9-7492-28df-11e6-cc0c3dc73e03,1
...,...,...,...,...,...
13170,2021-09-18,64580.5,11008787-72d8-8824-11de-d377741509e2,11008787-72d8-8824-11de-d377741509e2,0
13171,2021-08-24,56130.0,4a3c4086-7492-28df-11e5-b9d44d821aec,4a3c4086-7492-28df-11e5-b9d44d821aec,1
13172,2021-09-26,114940.0,4a3cc085-7492-28df-11e6-137b34be108f,4a3cc085-7492-28df-11e6-137b34be108f,1
13173,2021-08-27,52269.0,4a3ced86-7492-28df-11e6-dd53a25ffe10,4a3ced86-7492-28df-11e6-dd53a25ffe10,0


In [465]:
sum(df['Kol4'])

8196

In [ ]:
df
